
<center> <h1> Data Ingestion and cleaning <br> Pandas I/O and Data Cleaup and Preparation

</h1></center>

In [89]:
pandas_io_tables= pd.read_html('https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html')
len(pandas_io_tables)

1

In [90]:
pandas_io_tables[0]

,Format Type,Data Description,Reader,Writer
0,text,CSV,read_csv,to_csv
1,text,JSON,read_json,to_json
2,text,HTML,read_html,to_html
3,text,Local clipboard,read_clipboard,to_clipboard
4,binary,MS Excel,read_excel,to_excel
5,binary,HDF5 Format,read_hdf,to_hdf
6,binary,Feather Format,read_feather,to_feather
7,binary,Parquet Format,read_parquet,to_parquet
8,binary,Msgpack,read_msgpack,to_msgpack
9,binary,Stata,read_stata,to_stata


# read_csv

In [1]:
import pandas as pd


In [8]:
%%bash
echo -e  "a, b, c\n1, 3, 4\n11, 44, 55\n4, 5, 23"> sample_file.txxxt

In [9]:
! ls *.txxxt

sample_file.txxxt


In [13]:
!head sample_file.txxxt

a, b, c
1, 3, 4
11, 44, 55
4, 5, 23


This is a delimiter(comma separated) text file. So we can use pandas.read_csv

In [3]:
df= pd.read_csv('sample_file.txxxt')
df

,11,233,123
0,1,3,4
1,11,44,55
2,4,5,23


#  Earth Lab [Figshare](Figshare.com) repository.
download .csv containing monthly average precipitation for Boulder, CO

In [314]:
pd.read_csv('https://ndownloader.figshare.com/files/12710618')


,months,precip,seasons
0,Jan,0.70,Winter
1,Feb,0.75,Winter
2,Mar,1.85,Spring
3,Apr,2.93,Spring
4,May,3.05,Spring
5,June,2.02,Summer
6,July,1.93,Summer
7,Aug,1.62,Summer
8,Sept,1.84,Fall
9,Oct,1.31,Fall


read_csv has lot of options around **50** to handle complexity of  noisy csv files.

- CSV file has header missing(use **header = None** or give header names using **names = list of header names** )
- Skip some rows( use **skiprows= list of row number**)
- **na_values** to specify a **[ ] of NA values or a dictionary {column_name:na_value}**

In [16]:
%%bash
echo -e  "11, 233, 123\n1, 3, 4\n11, 44, 55\n4, 5, 23"> sample_file_no_header.txxxt
cat sample_file_no_header.txxxt

11, 233, 123
1, 3, 4
11, 44, 55
4, 5, 23


In [17]:
pd.read_csv('sample_file_no_header.txxxt', header= None)

,0,1,2
0,11,233,123
1,1,3,4
2,11,44,55
3,4,5,23


In [18]:
pd.read_csv('sample_file_no_header.txxxt', names= list('abc'))

,a,b,c
0,11,233,123
1,1,3,4
2,11,44,55
3,4,5,23


# read_table with regular expressions
example from the book

In [24]:
%%bash
echo -e '            A         B         C\n
 aaa -0.264438 -1.026059 -0.619500\n
 bbb  0.927272  0.302904 -0.032399\n
 ccc -0.264273 -0.386314 -0.217601\n
 ddd -0.871858 -0.348382  1.100491\n' > sample_file_variable_space.txxxt

In [315]:
!cat sample_file_variable_space.txxxt

            A         B         C

 aaa -0.264438 -1.026059 -0.619500

 bbb  0.927272  0.302904 -0.032399

 ccc -0.264273 -0.386314 -0.217601

 ddd -0.871858 -0.348382  1.100491



In [318]:
# What regex we should put in sep
pd.read_table('sample_file_variable_space.txxxt', sep=  )

# or read_csv

In [27]:
pd.read_csv('sample_file_variable_space.txxxt',sep=)

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


# Reading Text Files in Pieces

let dowload iris dataset form uci machine learning repository

In [319]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data

--2019-02-11 17:33:20--  https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.249
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4551 (4.4K) [text/plain]
Saving to: ‘iris.data.1’

iris.data.1         100%[===================>]   4.44K  --.-KB/s    in 0.003s  

2019-02-11 17:33:20 (1.54 MB/s) - ‘iris.data.1’ saved [4551/4551]



In [320]:
!head iris.data

5.1,3.5,1.4,0.2,Iris-setosa
4.9,3.0,1.4,0.2,Iris-setosa
4.7,3.2,1.3,0.2,Iris-setosa
4.6,3.1,1.5,0.2,Iris-setosa
5.0,3.6,1.4,0.2,Iris-setosa
5.4,3.9,1.7,0.4,Iris-setosa
4.6,3.4,1.4,0.3,Iris-setosa
5.0,3.4,1.5,0.2,Iris-setosa
4.4,2.9,1.4,0.2,Iris-setosa
4.9,3.1,1.5,0.1,Iris-setosa


Attribute Information:
   1. sepal length in cm
   2. sepal width in cm
   3. petal length in cm
   4. petal width in cm
   5. class: 
      -- Iris Setosa
      -- Iris Versicolour
      -- Iris Virginica


In [321]:
%%bash 
columns="sepal_length,sepal_width,petal_length,petal_width,class"
echo "$columns"| cat - iris.data > iris.csv

In [68]:
!head iris.csv

sepal_length,sepal_width,petal_length,petal_width,class
5.1,3.5,1.4,0.2,Iris-setosa
4.9,3.0,1.4,0.2,Iris-setosa
4.7,3.2,1.3,0.2,Iris-setosa
4.6,3.1,1.5,0.2,Iris-setosa
5.0,3.6,1.4,0.2,Iris-setosa
5.4,3.9,1.7,0.4,Iris-setosa
4.6,3.4,1.4,0.3,Iris-setosa
5.0,3.4,1.5,0.2,Iris-setosa
4.4,2.9,1.4,0.2,Iris-setosa


In [69]:
!wc -l iris.csv

152 iris.csv


Let's say this file is big and can't fit in memory. We can use **read_csv** with **chunksize** option.

Typically a summarization is performed in chunk to perform overall summarization.

In [322]:
chunks = pd.read_csv('iris.csv', chunksize=20)
print('type of chunks is {}'.format(type(chunks)))
total_sepal_length = 0
num_examples = 0
flower_by_type = pd.Series([])
for chunk in chunks:
    #??? check the type of chunk
    total_sepal_length += chunk['sepal_length'].sum()
    num_examples+= chunk.shape[0]
    
total_sepal_length/num_examples

type of chunks is <class 'pandas.io.parsers.TextFileReader'>


5.843333333333334

let's verify this

In [323]:
iris_df = pd.read_csv('iris.csv')
iris_df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


# Writing/converting data
use **to_** methods like
- to_csv
- to_pickle
- to_sql
- to_latex
- to_html
- t_excel
.
.
.

@TODO to_latex functionality

#  Handling JSON data
- Use pd.read_json to read json file
- use inbult json (loads, dumps) to handle json string and python object

In [324]:
iris_str_data = iris_df.to_json()
iris_str_data


'{"sepal_length":{"0":5.1,"1":4.9,"2":4.7,"3":4.6,"4":5.0,"5":5.4,"6":4.6,"7":5.0,"8":4.4,"9":4.9,"10":5.4,"11":4.8,"12":4.8,"13":4.3,"14":5.8,"15":5.7,"16":5.4,"17":5.1,"18":5.7,"19":5.1,"20":5.4,"21":5.1,"22":4.6,"23":5.1,"24":4.8,"25":5.0,"26":5.0,"27":5.2,"28":5.2,"29":4.7,"30":4.8,"31":5.4,"32":5.2,"33":5.5,"34":4.9,"35":5.0,"36":5.5,"37":4.9,"38":4.4,"39":5.1,"40":5.0,"41":4.5,"42":4.4,"43":5.0,"44":5.1,"45":4.8,"46":5.1,"47":4.6,"48":5.3,"49":5.0,"50":7.0,"51":6.4,"52":6.9,"53":5.5,"54":6.5,"55":5.7,"56":6.3,"57":4.9,"58":6.6,"59":5.2,"60":5.0,"61":5.9,"62":6.0,"63":6.1,"64":5.6,"65":6.7,"66":5.6,"67":5.8,"68":6.2,"69":5.6,"70":5.9,"71":6.1,"72":6.3,"73":6.1,"74":6.4,"75":6.6,"76":6.8,"77":6.7,"78":6.0,"79":5.7,"80":5.5,"81":5.5,"82":5.8,"83":6.0,"84":5.4,"85":6.0,"86":6.7,"87":6.3,"88":5.6,"89":5.5,"90":5.5,"91":6.1,"92":5.8,"93":5.0,"94":5.6,"95":5.7,"96":5.7,"97":6.2,"98":5.1,"99":5.7,"100":6.3,"101":5.8,"102":7.1,"103":6.3,"104":6.5,"105":7.6,"106":4.9,"107":7.3,"108":6.7,"1

use json(part of python standard library) to handle json related functionality

## Deserialize  str data to python object

In [325]:
import json
# Let's Deserialize iris_str_data
iris = json.loads(iris_str_data)
print(type(iris))
iris.keys()

<class 'dict'>


dict_keys(['class', 'petal_length', 'petal_width', 'sepal_width', 'sepal_length'])

## Convert python data to JSON

In [327]:
iris_json_obj = json.dumps(iris)
print(type(iris_json_obj))

<class 'str'>


# Binary dataformat
- pickle.
- HDF5(hierarchical data format) Format. 
    + Useful for large amount of data in data science/machine learning project. Allows efficient read and write for small portion of data. Relevent portion of data can be pulled into memory very fast(**some kind of queue mechanism**).

There are libraries in python(h5py) to handle hdf5 files but pandas has **HDFStore** interface.

In [328]:
# may have install tables. 
#pip install --upgrade tables
iris_h5_store = pd.HDFStore('iris_data.h5')

In [330]:
iris_h5_store['iris_class'] = iris_df['class'] 
iris_h5_store['iris_class']

0         Iris-setosa
1         Iris-setosa
2         Iris-setosa
3         Iris-setosa
4         Iris-setosa
5         Iris-setosa
6         Iris-setosa
7         Iris-setosa
8         Iris-setosa
9         Iris-setosa
10        Iris-setosa
11        Iris-setosa
12        Iris-setosa
13        Iris-setosa
14        Iris-setosa
15        Iris-setosa
16        Iris-setosa
17        Iris-setosa
18        Iris-setosa
19        Iris-setosa
20        Iris-setosa
21        Iris-setosa
22        Iris-setosa
23        Iris-setosa
24        Iris-setosa
25        Iris-setosa
26        Iris-setosa
27        Iris-setosa
28        Iris-setosa
29        Iris-setosa
            ...      
120    Iris-virginica
121    Iris-virginica
122    Iris-virginica
123    Iris-virginica
124    Iris-virginica
125    Iris-virginica
126    Iris-virginica
127    Iris-virginica
128    Iris-virginica
129    Iris-virginica
130    Iris-virginica
131    Iris-virginica
132    Iris-virginica
133    Iris-virginica
134    Iri

In [42]:
iris_h5_store.close()

To allow searching use foramt = table

In [340]:
iris_df.to_hdf('iris.h5', 'iris_table', format= 'table')

In [341]:
pd.read_hdf('iris.h5', 'iris_table', columns= ['sepal_length', 'class'])

,sepal_length,class
0,5.1,Iris-setosa
1,4.9,Iris-setosa
2,4.7,Iris-setosa
3,4.6,Iris-setosa
4,5.0,Iris-setosa
5,5.4,Iris-setosa
6,4.6,Iris-setosa
7,5.0,Iris-setosa
8,4.4,Iris-setosa
9,4.9,Iris-setosa


In [342]:
pd.read_hdf('iris.h5', 'iris_table', where = ['sepal_length<5.0'])

ValueError: The passed where expression: [sepal_length<5.0]
            contains an invalid variable reference
            all of the variable references must be a reference to
            an axis (e.g. 'index' or 'columns'), or a data_column
            The currently defined references are: columns,index


In [343]:
pd.read_hdf('iris.h5', 'iris_table', where = ['index < 10'])

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


In [345]:
iris_df.to_hdf('iris.h5', 'iris_table', format= 'table', data_columns=['sepal_length', 'class'])

/media/psnegi/psn/du/teaching/teach_2019/prep/data_science_tools1/prep_dst1_env/lib/python3.5/site-packages/tables/path.py:118: NaturalNameWarning: object name is a Python keyword: 'class'; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  % (name, _warnInfo), NaturalNameWarning)


In [346]:
pd.read_hdf('iris.h5', 'iris_table', where = ['sepal_length<5.0'])

,sepal_length,sepal_width,petal_length,petal_width,class
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa
11,4.8,3.4,1.6,0.2,Iris-setosa
12,4.8,3.0,1.4,0.1,Iris-setosa
13,4.3,3.0,1.1,0.1,Iris-setosa
22,4.6,3.6,1.0,0.2,Iris-setosa


# Data cleaning
## Handling Missing Data

In pandas missing data is NA(not available)..

- For numeric data, pandas uses the floating-point value NaN (Not a Number) to represent missing data.
- The built-in Python None value is also treated as NA


## APIs for handling NA
- **dropna:** Filter data based on axis labels, with varying thresholds for how much missing data to tolerate.
-  **fillna:** Fill NA with a value of some interpolation method	

## dropna

https://archive.ics.uci.edu/ml/machine-learning-databases/horse-colic/horse-colic.data

In [159]:
# As we talked earlier csv files have lot of options, can you figure out the option
#so that ? is treated as NA values
colic_horse_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/horse-colic/horse-colic.data', 
                   sep= '\s+', header= None)
colic_horse_df.sample(10)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
202,1.0,1,534073,37.5,48.0,40.0,NaN,NaN,NaN,NaN,...,41.0,55.0,3.0,2.0,3.0,1,2208,0,0,2
298,1.0,1,530612,36.5,100.0,24.0,3.0,3.0,3.0,1.0,...,50.0,6.0,3.0,3.4,1.0,1,2208,0,0,1
193,1.0,1,529373,NaN,60.0,48.0,3.0,3.0,4.0,2.0,...,58.0,7.6,NaN,NaN,2.0,1,3205,0,0,2
297,1.0,1,529386,37.5,72.0,30.0,4.0,3.0,4.0,1.0,...,60.0,6.8,NaN,NaN,2.0,1,3205,0,0,2
37,1.0,1,533847,37.8,72.0,NaN,NaN,3.0,NaN,1.0,...,56.0,80.0,1.0,2.0,1.0,1,3111,0,0,2
181,2.0,1,528305,37.8,48.0,14.0,NaN,NaN,1.0,1.0,...,35.0,7.5,NaN,NaN,1.0,2,3111,0,0,1
222,1.0,1,528800,NaN,112.0,13.0,4.0,4.0,4.0,1.0,...,60.0,6.3,3.0,NaN,1.0,1,2205,0,0,1
38,2.0,1,528996,38.6,52.0,NaN,1.0,1.0,1.0,1.0,...,32.0,6.6,1.0,5.0,1.0,2,0,0,0,1
267,2.0,1,527829,38.3,54.0,18.0,3.0,1.0,2.0,1.0,...,44.0,7.2,3.0,NaN,1.0,2,2124,0,0,1
13,2.0,9,5291329,38.0,92.0,28.0,1.0,1.0,2.0,1.0,...,37.0,6.1,1.0,NaN,2.0,2,0,0,0,1


In [103]:
colic_horse_df.dropna?

In [163]:
colic_horse_df.dropna(axis=0, how = 'any')


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
10,1.0,1,528548,38.1,66.0,12.0,3.0,3.0,5.0,1.0,...,44.0,6.0,2.0,3.6,1.0,1,2124,0,0,1
54,2.0,1,529461,40.3,114.0,36.0,3.0,3.0,1.0,2.0,...,57.0,8.1,3.0,4.5,3.0,1,7400,0,0,1
64,1.0,1,529667,39.0,64.0,36.0,3.0,1.0,4.0,2.0,...,44.0,7.5,3.0,5.0,1.0,1,2113,0,0,1
91,2.0,1,529461,40.3,114.0,36.0,3.0,3.0,1.0,2.0,...,57.0,8.1,3.0,4.5,2.0,1,3205,0,0,1
104,1.0,1,527563,37.8,52.0,24.0,1.0,3.0,3.0,1.0,...,48.0,6.6,1.0,3.7,2.0,1,5400,0,0,2
189,1.0,1,5299603,38.3,60.0,16.0,3.0,1.0,1.0,1.0,...,30.0,6.0,1.0,3.0,1.0,1,31110,0,0,2


**how = all** will drop if all row values or column values are NA

## same as this boolean indexing

In [157]:
import numpy as np
colic_horse_df[ colic_horse_df.notna().all(axis = 1)]

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
10,1.0,1,528548,38.1,66.0,12.0,3.0,3.0,5.0,1.0,...,44.0,6.0,2.0,3.6,1.0,1,2124,0,0,1
54,2.0,1,529461,40.3,114.0,36.0,3.0,3.0,1.0,2.0,...,57.0,8.1,3.0,4.5,3.0,1,7400,0,0,1
64,1.0,1,529667,39.0,64.0,36.0,3.0,1.0,4.0,2.0,...,44.0,7.5,3.0,5.0,1.0,1,2113,0,0,1
91,2.0,1,529461,40.3,114.0,36.0,3.0,3.0,1.0,2.0,...,57.0,8.1,3.0,4.5,2.0,1,3205,0,0,1
104,1.0,1,527563,37.8,52.0,24.0,1.0,3.0,3.0,1.0,...,48.0,6.6,1.0,3.7,2.0,1,5400,0,0,2
189,1.0,1,5299603,38.3,60.0,16.0,3.0,1.0,1.0,1.0,...,30.0,6.0,1.0,3.0,1.0,1,31110,0,0,2


In [156]:
# Keep  all the row with  number of na values < 4 ??


# Filling In Missing Data

In [3]:

np.random.seed(1)
df = pd.DataFrame(np.arange(21).reshape(7, 3))
df.iloc[1:4, 1] = np.nan
df.iloc[-2:, 2] = np.nan
df

,0,1,2
0,0,1.0,2.0
1,3,NaN,5.0
2,6,NaN,8.0
3,9,NaN,11.0
4,12,13.0,14.0
5,15,16.0,NaN
6,18,19.0,NaN


In [4]:
df.fillna({1: 0.5, 2: .5})

,0,1,2
0,0,1.0,2.0
1,3,0.5,5.0
2,6,0.5,8.0
3,9,0.5,11.0
4,12,13.0,14.0
5,15,16.0,0.5
6,18,19.0,0.5


In [180]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,0,1.0,2.0
1,3,1.0,5.0
2,6,1.0,8.0
3,9,NaN,11.0
4,12,13.0,14.0
5,15,16.0,14.0
6,18,19.0,14.0


In [181]:
# fill df with median value of each columns


# Data Transformation

# Dropping duplicates

In [185]:
df = pd.DataFrame(np.arange(12).reshape(4,3), columns=['col'+str(i) for i in range(1, 4)])
df

,col1,col2,col3
0,0,1,2
1,3,4,5
2,6,7,8
3,9,10,11


In [231]:
np.random.seed(1)
dup_df =pd.concat([df, df.sample(2)])
dup_df = dup_df.reset_index(drop= True)
dup_df.iloc[4, 0] = 33
dup_df

,col1,col2,col3
0,0,1,2
1,3,4,5
2,6,7,8
3,9,10,11
4,33,10,11
5,6,7,8


In [234]:
dup_df.duplicated(subset=['col1', 'col3'])

0    False
1    False
2    False
3    False
4    False
5     True
dtype: bool

In [236]:
dup_df.drop_duplicates(subset=['col1', 'col3'])

,col1,col2,col3
0,0,1,2
1,3,4,5
2,6,7,8
3,9,10,11
4,33,10,11


# Discretization and Binning

In [246]:
# exampel from the book
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]


In [240]:
cats = pd.cut(ages, bins)
print(cats)
type(cats)

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]


pandas.core.arrays.categorical.Categorical

In [244]:
print(cats.codes)
print(cats.categories)




[0 0 0 1 0 0 2 1 3 2 2 1]
IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')


bin counts

In [249]:
print(pd.value_counts(cats))

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64


## passing bin names

In [248]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

# outlier detection
- Unusual dollar amount
- Unusual pixel value in the image

build conditional logic to get boolean index

In [281]:
np.random.seed(1)
data = pd.DataFrame(2+ np.random.randn(10, 4))
data

,0,1,2,3
0,3.624345,1.388244,1.471828,0.927031
1,2.865408,-0.301539,3.744812,1.238793
2,2.319039,1.750630,3.462108,-0.060141
3,1.677583,1.615946,3.133769,0.900109
4,1.827572,1.122142,2.042214,2.582815
5,0.899381,3.144724,2.901591,2.502494
6,2.900856,1.316272,1.877110,1.064231
7,1.732112,2.530355,1.308339,1.603246
8,1.312827,1.154794,1.328754,1.987335
9,0.882690,2.234416,3.659802,2.742044


In [282]:
data.describe()

,0,1,2,3
count,10.000000,10.000000,10.000000,10.000000
mean,2.004181,1.595598,2.493033,1.548796
std,0.909089,0.933614,0.990475,0.902147
min,0.882690,-0.301539,1.308339,-0.060141
25%,1.404016,1.195164,1.573149,0.961331
50%,1.779842,1.502095,2.471902,1.421020
75%,2.728815,2.113469,3.380023,2.373705
max,3.624345,3.144724,3.744812,2.742044


In [283]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
0,3.624345,1.388244,1.471828,0.927031
1,2.865408,-0.301539,3.744812,1.238793
2,2.319039,1.750630,3.462108,-0.060141
3,1.677583,1.615946,3.133769,0.900109
5,0.899381,3.144724,2.901591,2.502494
9,0.882690,2.234416,3.659802,2.742044


In [284]:
data[0].quantile(q= .99)

3.559231316367347

In [285]:
data[0][np.abs(data[0]) > data[0].quantile(q= .99)] = data[0].quantile(q= .99)
data

,0,1,2,3
0,3.559231,1.388244,1.471828,0.927031
1,2.865408,-0.301539,3.744812,1.238793
2,2.319039,1.750630,3.462108,-0.060141
3,1.677583,1.615946,3.133769,0.900109
4,1.827572,1.122142,2.042214,2.582815
5,0.899381,3.144724,2.901591,2.502494
6,2.900856,1.316272,1.877110,1.064231
7,1.732112,2.530355,1.308339,1.603246
8,1.312827,1.154794,1.328754,1.987335
9,0.882690,2.234416,3.659802,2.742044


# Computing Indicator/Dummy Variables

converting a categorical variable into a “dummy” or “indicator” matrix

Check the book too

In [289]:
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

--2019-02-11 16:56:56--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.34.235
Connecting to files.grouplens.org (files.grouplens.org)|128.101.34.235|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip.1’

ml-latest-small.zip 100%[===================>] 955.28K  1.05MB/s    in 0.9s    

2019-02-11 16:56:57 (1.05 MB/s) - ‘ml-latest-small.zip.1’ saved [978202/978202]



In [290]:
!unzip ml-latest-small.zip


Archive:  ml-latest-small.zip
replace ml-latest-small/links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [293]:
!head ml-latest-small/movies.csv

In [298]:
movie_df = pd.read_csv('ml-latest-small/movies.csv', index_col = 0)
movie_df.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [308]:
unique_genre =  pd.unique(movie_df.genres.str.split('|').sum())
unique_genre

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'War', 'Musical', 'Documentary', 'IMAX',
       'Western', 'Film-Noir', '(no genres listed)'], dtype=object)

In [310]:
# build columns for different genre indicator

In [313]:
movie_df.head()

,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,True,True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Jumanji (1995),Adventure|Children|Fantasy,True,False,True,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Grumpier Old Men (1995),Comedy|Romance,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Waiting to Exhale (1995),Comedy|Drama|Romance,False,False,False,True,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
5,Father of the Bride Part II (1995),Comedy,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
